***Aviso***: 

Este jupyter notebook foi adaptado a partir da versão de Michael Dobrzanski, disponível em <br>
https://github.com/MichalDanielDobrzanski/DeepLearningPython35

que por sua vez é uma implementação para python3 do código de Michael Nielsen, disponível em <br>
https://github.com/mnielsen/neural-networks-and-deep-learning


### 1 - Objetivo

A intenção deste exercício é entender como se funciona o algoritmo de *Backpropagation*, maior responsável pelo sucesso das redes neurais artificiais. Nós vamos estudar como implementar o algoritmo com a intenção de treinarmos uma rede neural para classificar imagens de números do dataset [MNIST](http://yann.lecun.com/exdb/mnist/).

Para tal, nós vamos utilizar o código disponibilizado por Michael Nielsen, autor do livro ["*Neural Networks and Deep Learning*"](http://neuralnetworksanddeeplearning.com/). Parte do laço de backpropagation foi removido e a tarefa será completar esse laço. 



### 2 - Carregando as bibliotecas

Nós vamos utilizar funções básicas do python junto com a biblioteca NumPy. Esta biblioteca é otimizada para operações de álgebra linear e é base de quase todas as bibliotecas matemáticas disponíveis para o python (inclusive o scikit-learn que utilizamos anteriormente e o tensorflow que será utilizado futuramente).




In [ ]:
# Standard library
import random
import pickle
import gzip

# Third-party libraries
import numpy as np

### 3 - Funções auxiliares

Para facilitar o exercício, algumas funções auxiliares estão disponíveis abaixo.


Primeiro, temos funções que irão nos ajudar carregando o dataset e dividindo-o em treino, validação e teste.

In [ ]:
def load_data():
    f = gzip.open('data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper(): 
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

training_data, validation_data, test_data = load_data_wrapper()

Tendo carregado os datasets, precisamos implementar as funções de ativação dos neurônios artificiais e a sua derivada para utilizar gradiente descendente e backpropagation. 

As funções python abaixo são implementações da função sigmoid (```sigmoid```) e de sua derivativa (```sigmoid_prime```).

$
\begin{align}
\sigma(z) &= \frac{1}{1 + e^{-z}} \nonumber \\
 \nonumber \\
\sigma^{\prime}(z) &= \sigma(z) * (1 - \sigma(z)) \nonumber
\end{align}
$






In [ ]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

### 3 - O laço de backpropagation

1. Entrada: calcular a ativação para a camada de entrada<br>
    <br> $a^1 = x \nonumber $ <br>
    <br>
2. Feedforward: Para cada $l = 2, 3, \dots,L$ <br>
    <br>$ \begin{align} 
    z^{l} &= w^l a^{l-1}+b^l \nonumber \\
    \nonumber \\
    a^{l} &= \sigma(z^{l}) \nonumber  \\
    \nonumber \\
    \end{align} $
<br>
3. Calcular erro na saída <br>

    $ \begin{align}   
    \delta^{L} = \nabla_a C \odot \sigma^{\prime}(z^L) \nonumber \\
    \nonumber \\
    \end{align} $

4. Backpropagate: para cada $l = L-1, L-2, \dots, 2$

    <br>$ \begin{align}   
    \delta^{l} &= ((\mathbf{W}^{l+1})^T \delta^{l+1}) \odot \sigma^{\prime}(z^{l}) \nonumber \\
    \nonumber \\    
    \frac{\partial C}{\partial w^l_{jk}} &= a^{l-1}_k \delta^l_j \nonumber \\
    \frac{\partial C}{\partial b^l_j} &= \delta^l_j \nonumber
    \end{align} $<br>


### 4 - Implementando o loop de backprop

Este desafio irá demonstrar o quanto você conseguiu entender do algoritmo de backpropagation. A maior parte do código já esta implementada na classe Network abaixo (etapas 1, 2, 3 e o cálculo do $\delta^{l}$ da etapa 4). O seu trabalho é completar o laço ```for``` que está em branco.

Algumas dicas do que observar:

1. As ativações do forward de cada camada estão salvas na variável ```zs```.
2. ```sigmoid_prime``` devolve a derivada da função sigmoid.
3. Para implementar a função XXXXXX, use a função ```np.dot``` da biblioteca numpy.
4. O utilizar-se um índice negativo em uma lista em python, o mesmo seleciona um item de trás para frente. Por exemplo: 
```
l = [a, b, c, d, e]
l[-2] == "d"
```
5. Para obter uma matriz transposta a partir de um array da biblioteca numpy, basta chamar a função transpose(). Por exemplo:
```
nome_do_array.transpose()
```


***Dica***: o código dentro do laço pode ser escrito em apenas 5 linhas de código!

In [ ]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):        
        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test));
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, lr):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(lr/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(lr/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        
        for l in range(2, self.num_layers):
            pass
        
            #######    

            # implemente seu código aqui            

            #####
        
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)


### 4 - Definindo a rede neural

A classe ```Network``` vai criar a rede neural para treinarmos. Ela recebe como parâmetro uma lista contendo os tamanhos das camadas de nossa rede neural. Para cada valor que passarmos nesta lista, uma camada com o número de neurônios correspondente será adicionada ao modelo.

Por exemplo

```python
net = Network([784, 30, 10])
```

define uma rede neural com 3 camadas e tamanhos 784, 30 e 10 respectivamente.

---
*Observação 1: o primeiro valor desta lista deve ter, obrigatoriamente, o tamanho do vetor de variáveis* ($784$).<br>
*Observação 2: o último valor desta lista deve corresponder ao número de classes que serão classificadas* ($10$).

In [ ]:
net = Network([784, 30, 10])

### 5 - Treinando a rede neural

Uma vez definida, devemos começar o processo de treinamento da rede neural. Para isso, a classe ```Network``` define uma função chamada ```SGD``` que vai tomar conta dos processo de ```forward``` e ```backward``` executando o algoritmo de backpropagation e ao mesmo tempo atualizar os parâmetros da rede neural usando gradiente descendente.

Observação 3: chame esta função apenas depois que você tiver terminado a implementação do laço ```for``` na sessão 3.

In [ ]:
net.SGD(training_data=training_data, 
        epochs=30, 
        mini_batch_size=10, 
        eta=3.0, 
        test_data=test_data)

Observe como o percentual de acertos começa abaixo de $90\%$ e vai subindo até estabilizar por volta de $94\%$ (resultados podem variar devido à inicialização randômica dos parâmetros - pesos).

Note também como o percentual de algumas das épocas anteriores é maior do que o percentual de acertos de épocas mais recentes. Nós vamos estudar uma forma de tirar proveito disto na próxima lição.